<a href="https://colab.research.google.com/github/cwattsnogueira/rating-predictor-spam-detection-review-summarizer/blob/main/09_ReviewAssistantBERT5_ClassModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Upload your ZIP file (bert_5class_rating_model.zip)
from google.colab import files
uploaded = files.upload()

# Step 2: Extract filename
import os
zip_filename = next(iter(uploaded))  # gets the first uploaded file

# Step 3: Create target folder
target_folder = "/content/"
os.makedirs(target_folder, exist_ok=True)

# Step 4: Unzip into the folder
import zipfile
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(target_folder)

print(f" Unzipped '{zip_filename}' into '{target_folder}'")

Saving bert_5class_rating_model.zip to bert_5class_rating_model.zip
 Unzipped 'bert_5class_rating_model.zip' into '/content/'


In [2]:
# Step 1: Upload your ZIP file (imagens.zip)
from google.colab import files
uploaded = files.upload()

# Step 2: Extract filename
import os
zip_filename = next(iter(uploaded))  # gets the first uploaded file

# Step 3: Create target folder
target_folder = "/content/"
os.makedirs(target_folder, exist_ok=True)

# Step 4: Unzip into the folder
import zipfile
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(target_folder)

print(f" Unzipped '{zip_filename}' into '{target_folder}'")

Saving imagens.zip to imagens.zip
 Unzipped 'imagens.zip' into '/content/'


In [3]:
!pip install gradio transformers --quiet

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.nn.functional import softmax
import gradio as gr

In [5]:
# Load fine-tuned 5-class BERT model
model = BertForSequenceClassification.from_pretrained("/content/bert_5class_rating_model")
tokenizer = BertTokenizer.from_pretrained("/content/bert_5class_rating_model")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [6]:
# Render stars visually using emojis
def render_stars(n):
    return " ".join(["⭐" if i < n else "☆" for i in range(5)])

In [7]:
# Main prediction function
def estimate_star_score_with_feedback(product, title, review, image=None):
    if not title.strip() and not review.strip():
        return "⚠️ Please enter both a title and review before submitting."

    combined_text = f"{title}. {review}"
    inputs = tokenizer(combined_text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)

    with torch.no_grad():
        logits = model(**inputs).logits
        probs = softmax(logits, dim=-1)[0]  # shape [5]
        pred_class = torch.argmax(probs).item()  # 0–4
        stars = pred_class + 1
        confidence = probs[pred_class].item()

        visual = render_stars(stars)

        breakdown = "\n".join([f"- {i+1}⭐: {probs[i].item():.2%}" for i in range(5)])

        warning = ""
        if confidence < 0.6:
            warning = "\n⚠️ This prediction has low confidence. Consider reviewing manually."

    response = f"""
### Thank you for reviewing **{product}**!

**Review Title:** {title}

Our AI analyzed your review using a fine-tuned BERT model trained on customer experiences.

---

**Predicted Rating:** {stars}
{visual}
**Model Confidence:** {confidence:.2%}

**Confidence Breakdown:**
{breakdown}

{warning}

---
"""
    return response

In [8]:
# Example product list
product_list = {
    "Select your product": "Your Product Here",
    "Olay Regenerist Deep Hydration Regenerating Cream": "/content/imagens/olay_regenerist.png",
    "Heinz Tomato Ketchup, 38oz": "/content/imagens/heinz_ketchup.png",
    "Pantene Color Preserve Volume Shampoo, 25.4oz": "/content/imagens/pantene_color_preserve.png",
    "Kind Dark Chocolate Chunk Gluten Free Granola Bars - 5 Count": "/content/imagens/kind_granola_bars.png",
    "Johnson's Baby Bubble Bath and Wash, 15oz": "/content/imagens/johnsons_bubble_bath.png",
    "Windex Original Glass Cleaner Refill 67.6oz (2 Liter)": "/content/imagens/windex_glass_cleaner.png",
    "CeraVe SA Renewing Cream": "/content/imagens/cerave_sa_cream.png",
    "Disney Jake And The Neverland Pirates 4 Piece Bedding Set - Toddler": "/content/imagens/jake_pirates_bedding.png",
    "Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total": "/content/imagens/clorox_wipes.png",
    "Pink Friday: Roman Reloaded Re-Up (w/dvd)": "/content/imagens/nicki_minaj_album.png"
}

In [9]:
# Gradio UI
with gr.Blocks(title="Retail Review Assistant (BERT 5-Class Model)") as demo:
    gr.Markdown("### 🏬 Retailers You’ve Purchased From")
    with gr.Row():
        gr.Image(value="/content/imagens/walmart_logo.png", show_label=False, width=150, height=60)
        gr.Image(value="/content/imagens/target_logo.png", show_label=False, width=150, height=60)
        gr.Image(value="/content/imagens/amazon_logo.png", show_label=False, width=150, height=60)

    gr.Markdown("## 👋 Welcome back, Carllos!\nLeave a review for a product you've recently purchased.")

    gr.Markdown("### 🛒 Products You’ve Purchased")
    with gr.Row():
        product_dropdown = gr.Dropdown(choices=list(product_list.keys()), label="Select a Product", interactive=True)
        product_image = gr.Image(label="Product Preview", width=250, height=250, show_label=False)

    def update_image(product_name):
        return product_list.get(product_name, None)

    product_dropdown.change(fn=update_image, inputs=product_dropdown, outputs=product_image)

    review_title = gr.Textbox(lines=1, label="Review Title")
    review_input = gr.Textbox(lines=5, label="Write your review")
    submit_btn = gr.Button("Submit Review")
    output = gr.Markdown(label="Review Summary")

    submit_btn.click(fn=estimate_star_score_with_feedback,
                     inputs=[product_dropdown, review_title, review_input],
                     outputs=output)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6d38d262246fce5d93.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
